In [1]:
import pandas as pd
import os
from datetime import datetime

diretorio_origem = "raw-data/all-data"
diretorio_destino = "clear-data"

tabela_horarios = pd.read_excel("bcw-operation-calendar.xlsx")
tabela_horarios["date"] = pd.to_datetime(tabela_horarios["date"]).dt.date

# Lista todos os arquivos CSV na pasta de origem
arquivos = os.listdir(diretorio_origem)

# Lista para armazenar os DataFrames tratados
dataframes_tratados = []

for arquivo in arquivos:
    if arquivo.endswith(".csv"):
        caminho_arquivo_origem = os.path.join(diretorio_origem, arquivo)
        df = pd.read_csv(caminho_arquivo_origem)
        
        # Renomear as colunas
        df.rename(columns={"Ride": "ride_name", "Date/Time": "data_hora", "Wait Time": "wait_time"}, inplace=True)
        
        # Separar a coluna 'data_hora' em colunas de 'date' e 'time'
        df["date"] = pd.to_datetime(df["data_hora"]).dt.date
        df["time"] = pd.to_datetime(df["data_hora"]).dt.time
        
        # Inicializar uma lista para armazenar as linhas que atendem aos critérios
        linhas_mantidas = []
        
        for indice, linha in df.iterrows():            
            # Encontrar os horários de abertura e fechamento para o dia atual
            horarios_do_dia = tabela_horarios[tabela_horarios["date"] == linha["date"]]
            
            if not horarios_do_dia.empty:
                abertura = horarios_do_dia["park_opening_time"].iloc[0]
                fechamento = horarios_do_dia["park_closing_time"].iloc[0]
                parque_aberto = horarios_do_dia["park_open"].iloc[0]
                
                if parque_aberto:
                    if abertura is not None and fechamento is not None:
                        # Verificar se a hora está dentro do intervalo de abertura e fechamento
                        if abertura <= linha["time"] <= fechamento:
                            linhas_mantidas.append(indice)

        # Verificar se há linhas mantidas antes de prosseguir
        if len(linhas_mantidas) > 0:
            # Criar um novo DataFrame com as linhas que atendem aos critérios
            df_filtrado = df.loc[linhas_mantidas]
            
            # Criar um novo DataFrame com as colunas desejadas da tabela_horarios
            colunas_desejadas = ["date", "day_of_the_week", "sale", "holiday", "holiday_season", "season",
                                 "HotWheels 1130", "HotWheels 1330", "HotWheels 1530", "HotWheels 1730", "Acqua 1400",
                                 "Acqua 1500", "Madagascar 1500", "Madagascar 1600", "Madagascar 1700", "Sonho 1800",
                                 "Excalibur 1200", "Excalibur 1430", "Excalibur 1700"]
            
            # Filtrar as colunas desejadas da tabela_horarios correspondentes à data do arquivo atual
            colunas_desejadas_df = tabela_horarios[tabela_horarios["date"] == df_filtrado["date"].iloc[0]][colunas_desejadas]
            
            # Mesclar o dataframe filtrado com as colunas desejadas
            df_filtrado = pd.merge(df_filtrado, colunas_desejadas_df, on="date", how="left")
            
            # Adicionar o DataFrame tratado à lista de DataFrames tratados
            dataframes_tratados.append(df_filtrado)
            
            print("TERMINADO O ARQUIVO " + caminho_arquivo_origem)

# Concatenar todos os DataFrames tratados em um único DataFrame
df_final = pd.concat(dataframes_tratados, ignore_index=True)

# Salvar o DataFrame final em um único arquivo
caminho_arquivo_destino = os.path.join(diretorio_destino, "dados_tratados.csv")
df_final.to_csv(caminho_arquivo_destino, index=False)

print("TERMINADO O PROCESSAMENTO DE TODOS OS ARQUIVOS")


TERMINADO O ARQUIVO raw-data/all-data\autopista-01-jan.csv
TERMINADO O ARQUIVO raw-data/all-data\autopista-02-feb.csv
TERMINADO O ARQUIVO raw-data/all-data\autopista-03-mar.csv
TERMINADO O ARQUIVO raw-data/all-data\autopista-04-apr.csv
TERMINADO O ARQUIVO raw-data/all-data\autopista-05-may.csv
TERMINADO O ARQUIVO raw-data/all-data\autopista-06-jun.csv
TERMINADO O ARQUIVO raw-data/all-data\autopista-07-jul.csv
TERMINADO O ARQUIVO raw-data/all-data\autopista-08-aug.csv
TERMINADO O ARQUIVO raw-data/all-data\autopista-09-sep.csv
TERMINADO O ARQUIVO raw-data/all-data\autopista-10-oct.csv
TERMINADO O ARQUIVO raw-data/all-data\autopista-11-nov.csv
TERMINADO O ARQUIVO raw-data/all-data\autopista-12-dec.csv
TERMINADO O ARQUIVO raw-data/all-data\babyelefante-01-jan.csv
TERMINADO O ARQUIVO raw-data/all-data\babyelefante-02-feb.csv


KeyboardInterrupt: 